In [15]:

import requests
from bs4 import BeautifulSoup
import pandas as pd
import time
from urllib.parse import urljoin
from datetime import datetime
headers = {
   "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/138.0.0.0 Safari/537.36"
}

# 🔧 Fonction pour extraire nom, spécialité et ville
def extract_nom_specialite_ville(alt_text):
    alt_text = alt_text.strip()
    parts = alt_text.split(',')
    if len(parts) < 3:
        return "", "", ""
    nom = parts[0].strip()
    ville = parts[-1].strip()
    specialite = ','.join(parts[1:-1]).strip()
    return nom, specialite, ville

# 
donnees_medecins = []
date=datetime.now()

#  Boucle sur les pages
for page in range(1, 1286):
    url = f"https://www.dabadoc.com/recherche/page/{page}?button=&country=MA" if page > 1 else "https://www.dabadoc.com/recherche?button=&country=MA"
    print(f"\n🔎 Page {page} → {url}")
    response = requests.get(url, headers=headers)
    if response.status_code != 200:
        print(f"Erreur de chargement : {response.status_code}")
        continue

    soup = BeautifulSoup(response.text, "html.parser")
    blocs = soup.find_all("div", class_="search_doc_row")
    print(f" Médecins trouvés : {len(blocs)}")

    for bloc in blocs:
        try:
            #Accéder au profil
            a_tag = bloc.find("a", href=True)
            profil_link = urljoin("https://www.dabadoc.com", a_tag['href']) if a_tag else ""
            
            result_box = bloc.find("div", class_="result-box rounded")
            identifiant = result_box.get("id") if result_box else ""
            img_tag = bloc.find("img", alt=True, attrs={"data-src": True})
            alt_text = img_tag['alt'] if img_tag else "Non trouvé"
            photo_url = img_tag['data-src'] if img_tag else "Aucune"
            nom, specialite, ville = extract_nom_specialite_ville(alt_text)

            statut = "Fictive"
           

            horaire, telephone, adresse = "", "", ""

            if bloc.find("div", class_="availabilities-box"):
                    statut = "Actif"
            elif "Prendre RDV en Video" in bloc.text or bloc.find("a", class_="btn btn-success mt-1 text-white"):
                    statut = "Passif"
            elif bloc.find("div",class_="no-appointments-msg"):
                    statut = "Fictif"
                

            #  Scraping du profil pour plus de détails
            
            print(f" Profil : {profil_link}")
            try:
             profil_res = requests.get(profil_link, headers=headers)
             if profil_res.status_code == 200:
                profil_soup = BeautifulSoup(profil_res.text, "html.parser")
                adresse_div = profil_soup.find("div", class_="card-text")
                if adresse_div:
                        adresse = adresse_div.get_text(strip=True, separator=' ')
                #Extraire les actes     
                SoinsActes = []
                soins_badges = profil_soup.find_all("a", class_="badge badge-secondary p-2 mb-1")
                for soin in soins_badges:
                    texte = soin.get_text(strip=True)
                    if texte and isinstance(texte, str):
                        SoinsActes.append(texte)
                #Extraire les diplomes,certificats et formations à la fois
                diplomes = []
                for balise in profil_soup.find_all("i", class_="fa fa-circle"):
                    texte = balise.next_sibling
                    if texte and isinstance(texte, str):
                        diplomes.append(texte.strip()) 
                #Extraire le numero de téléphone
                phone_btn = profil_soup.find("a", id="phone-number-btn")
                if phone_btn and "href" in phone_btn.attrs:
                        tel_href = phone_btn["href"]
                        if tel_href.startswith("tel:"):
                                telephone = tel_href.replace("tel:", "").strip()
                #Extraire les langues
                Langues_Parlees = ""
                icone_langues = profil_soup.find("i", class_="fa fa-language")
                if icone_langues:
                    div_card_text = icone_langues.find_next("div", class_="card-text")
                    if div_card_text:
                        Langues_Parlees = div_card_text.get_text(strip=True)
               
            except Exception as e:
                    print(" Erreur profil :", e)
            
            # Ajout dans la liste
            donnees_medecins.append({
                "ID Medecin": identifiant,
                "Nom Complet": nom,
                "Spécialité": specialite,
                "Ville": ville,
                "Lien Profil": profil_link,
                "Photo URL": photo_url,
                "Statut": statut,
                "Téléphone": telephone,
                "Adresse": adresse,
                "Diplomes et Formations":diplomes,
                "Date Extraction" : date,
                "Soins et Actes":SoinsActes,
                "Langues Parlées":Langues_Parlees
            })

            print( identifiant, "|", specialite, "|", ville, "|", statut, "|", adresse,"|",datetime.now(),"|",diplomes,"|",SoinsActes,"|",Langues_Parlees)
        except Exception as err:
            print(" Erreur bloc médecin :", err)

    time.sleep(1.5)  # Pause pour éviter le blocage IP

# Export vers Excel 
df = pd.DataFrame(donnees_medecins)
df.to_excel(r"C:\Users\HP\Desktop\Data_Folder\Fichier_Dabadoc_ma_Avant_Nettoyage.xlsx", index=False)
print("\n Export terminé ! Nombre total de médecins :", len(df))




🔎 Page 1 → https://www.dabadoc.com/recherche?button=&country=MA
 Médecins trouvés : 15
 Profil : https://www.dabadoc.com/ma/gynecologue/casablanca/fatima-azzahra-kenkour
search-result-5aaade614578b300045fcf2d | Gynécologue, Gynécologue-obstétricien | Casablanca | Actif | Lot Al Madina Imm (J), Er étage, Av Abou Baker El Kadiri, Sidi Maarouf, Sidi Maarouf, Casablanca, Maroc | 2025-07-29 15:11:28.772744 | ['Cabinet de Gynécologie-Obstétrique Dr Kenkour Fatim-Azzahra.', 'Spécialiste en Gynécologie-Obstétrique.', 'Lauréate de la faculté de médecine et de pharmacie de Casablanca.', 'Diplômée de l\'Université " Paris Cité "'] | ["Ablation d'un kyste ovarien", "Ablation d'un nodule du sein", 'Ablation de norplant', 'Ablation de polype utérin', 'Ablation de stérilet', 'Accouchement', 'Accouchement par césarienne', 'Annexectomie', 'Aspiration endo-utérine', 'Assistance médicale à la procréation', "Cancer de l'utérus", 'Chirurgie Sénologique', "Chirurgie de l'incontinence urinaire", "Chirurgie 

KeyboardInterrupt: 

In [22]:

#Nettoyage avec Pandas:
#On va travailler avec df1 et non pas df pour garder la version originale dans df
#Lors de la phase de nettoyage on va se concentrer sur le nettoyage des elements de notre shema global
import pandas as pd,re
df1= df
#--------------------------Extraire le dernier Tiret Pour Avoir uniquement le ID sous forme d'un nombre sans serach -----------------------------
df1["ID Medecin"] = df1["ID Medecin"].str.rsplit("-", n=1).str[-1]
# ---------------------------Extraire la Civilité-------------------------------------------------
civilites = ["Dr", "dr", "Mme", "Mlle", "Pr", "Prof", "professeur"]
df1["Civilité"] = df1["Nom Complet"].str.strip().str.split().str[0]
df1["Civilité"] = df1["Civilité"].where(df1["Civilité"].isin(civilites), "")
# Supprimer la civilité dans le nom complet
df1["Nom Complet"] = df1.apply(
    lambda row: row["Nom Complet"].replace(row["Civilité"], "", 1).strip() if row["Civilité"] else row["Nom Complet"],
    axis=1
)
#-------------Nettoyer Nom Complet---------------------------------------
PRENOMS_COMPOSES = [
    "Fati Zahra",
    "Fatim Azzahra",
    "Fatima Azzahra",
    "Fatima Ezzahra",
    "Fatima Zahra",
    "Mohammed Amine",
    "Mohamed Amine",
    "Abdel Hakim",
    "Jean Pierre"
]
MOTS_CATEGORIES = ["centre", "cabinet", "laboratoire", "clinique", "center","Laser"]
MOTS_PARASITES=["Pr","Mme","Mr","Dr","Kiné","Quiétude","Mlle"," Dental","Psychologue"]
PRENOMS_CONNUS = ["Ouidad", "ahmed", "meryem", "imane", "fatima", "youssef","ahlam"]
patternMotParasites = r"\b(" + "|".join(MOTS_PARASITES) + r")\b"
#Nettoyer les mots indésirables avant de séparer le nom complet
def nettoyer_nom_complet(nom):
    nom = nom.strip()
    if not nom:
        return "", "Inconnu"

    # Normalisation des espaces
    nom = re.sub(r"\s+", " ", nom)

    # --- CAS PARTICULIER : "Dr Laboratoire ..." ---
    # Si on trouve Dr suivi d'un mot d'établissement
    if re.search(r"\bDr\s+(Laboratoire|Centre|Cabinet|Clinique|Center)\b", nom, flags=re.IGNORECASE):
        # On garde tout ce qui est après l'établissement
        for mot in MOTS_CATEGORIES:
            pattern = re.search(rf"\b{mot}\b", nom, flags=re.IGNORECASE)
            if pattern:
                # Tout ce qui est après le mot clé
                nom_medecin = nom[pattern.end():].strip()
                for prenom in PRENOMS_CONNUS:
                    nom_medecin=re.search(rf"\b{prenom}\b", nom, flags=re.IGNORECASE)
                    if nom_medecin:
                        reste = nom[nom_medecin.start():].strip()
                        reste = re.sub(patternMotParasites, "", reste, flags=re.IGNORECASE).strip()
                        return reste, "Medecin"
                return nom_medecin, "Medecin"
    # --- LOGIQUE CLASSIQUE (autres cas) ---
    prefix_match = re.search(r"\b(Dr|Mme|Mlle|Mr|M|Pr)\b", nom, flags=re.IGNORECASE)
    # Vérifier si un mot-clé d’établissement est présent
    for mot in MOTS_CATEGORIES:
        pattern = re.search(rf"\b{mot}\b", nom, flags=re.IGNORECASE)
        if pattern:
            if prefix_match and prefix_match.start() > pattern.start():
                nom_medecin = nom[prefix_match.end():].strip()
                nom_medecin = re.sub(patternMotParasites, "", nom_medecin, flags=re.IGNORECASE).strip()
                return nom_medecin, "Medecin"
            elif prefix_match and prefix_match.start() < pattern.start():
                nom_before = nom[:pattern.start()].strip()
                nom_before = re.sub(patternMotParasites, "", nom_before, flags=re.IGNORECASE).strip()
                return nom_before, "Medecin"
            else:
                nom_medecin = nom[pattern.end():].strip()
                for prenom in PRENOMS_CONNUS:
                # Supprimer le mot-clé d'établissement et titres
                    match_prenom=re.search(rf"\b{prenom}\b", nom_medecin, flags=re.IGNORECASE)
                    if match_prenom:
                        reste = nom_medecin[match_prenom.start():].strip()
                        reste = re.sub(patternMotParasites, "", reste, flags=re.IGNORECASE).strip()
                        return reste, "Medecin"
                return nom, "Etablissement"
    # Aucun mot-clé d’établissement
    nom_clean = re.sub(patternMotParasites, "", nom, flags=re.IGNORECASE).strip()
    return nom_clean, "Medecin"

def split_nom_complet(nom_complet):
    if not nom_complet:
        return "", ""
    nom_complet = nom_complet.title().strip().replace(".","")
    mots = nom_complet.split()
    if len(mots) == 1:
        return mots[0], ""
    titres = ("El", "Al", "Lalla", "Sidi", "Moulay", "Mly","Le")
    prenom = ""
    reste = []
    # Vérifier si le premier mot est un titre
    if mots[0] in titres:
        prenom = mots[0]
        reste = mots[1:]
        for i in range(2, 4):  # tester 2 ou 3 mots
            if " ".join(reste[:i]) in PRENOMS_COMPOSES:
                prenom = (prenom + " " + " ".join(reste[:i])).strip()
                nom = " ".join(reste[i:])
                return prenom, nom
        if prenom:
            prenom = (prenom + " " + reste[0]).strip()
            nom = " ".join(reste[1:])
            return prenom, nom
    else:
        reste = mots
        for i in range(2, 4):  # tester 2 ou 3 mots
            if " ".join(reste[:i]) in PRENOMS_COMPOSES:
                prenom = " ".join(reste[:i]).strip()
                nom = " ".join(reste[i:])
                return prenom, nom
  
    return reste[0], " ".join(reste[1:])
def supprimer_doublons(texte):
    mots = str(texte).split()
    resultat = []
    for mot in mots:
        if not resultat or resultat[-1] != mot:
            resultat.append(mot)
    return " ".join(resultat)
df1[["Nom Nettoyé","Type"]]=df1["Nom Complet"].apply(lambda x: pd.Series(nettoyer_nom_complet(x)))
df1[["Prénom","Nom"]]=df1["Nom Nettoyé"].apply(lambda x:pd.Series(split_nom_complet(x)))
df1["Prénom"]=df1["Prénom"].astype(str).str.title().str.strip().str.replace("-"," ").apply(supprimer_doublons)
df1["Nom"]=df1["Nom"].astype(str).str.title().str.strip().str.replace("-"," ").apply(supprimer_doublons)
#------------------Netoyer Spécialité-----------------------------
#Prendre uniquement la première Spécialité---------------
df1["Spécialité"]=df1["Spécialité"].astype(str).apply(lambda x: x.split(",")[0].strip())
classification = {
    "Anesthésiologie": [
        "réanimateur pédiatrique","réanimateur médical","anesthésie", "anesthésie-réanimation","anesthésiste-réanimateur"
    ],
    "Andrologie": [
        "andrologie","andrologue"
    ],
    "Cardiologie": [
        "cardiologie", "cardiologue", "cardiologie interventionnelle", "rythmologie interventionnelle","rythmologue interventionnel"
    ],
    "Chirurgie": [
        "chirurgie" ,"proctologie","proctologue"
    ],
    "Chirurgie cardiaque": [
        "chirurgie cardio", "chirurgie cardio-vasculaire"
    ],
    "Chirugie esthétique, plastique et reconstructive": [
        "chirurgie esthétique", "chirurgie plastique", "chirurgie réparatrice","médecin esthétique",
        "chirurgie plastique, reconstructrice et esthétique","médecine esthétique","médecine au laser"
    ],
    "Chirurgie générale": [
        "chirurgie générale", "chirurgie general", "chirurgie digestive", "chirurgie cancérologique","Proctologie"
    ],
    "Chirurgie maxillo-faciale": [
        "chirugien cervico-facial","stomatologue","chirurgie maxillo", "chirurgie cervico-faciale", "oto-rhino-laryngologie et chirurgie cervico-faciale","Chirugie Cervico-Faciale","chirurgien cervico-facial"
    ],
    "Chirurgie pédiatrique": [
        "chirurgie pédiatrique"
    ],
    "Chirurgie thoracique": [
        # à compléter si des mots-clés spécifiques apparaissent dans tes données
    ],
    "Chirurgie vasculaire": [
        "phlébologue","chirurgie vasculaire","Phlébologie","angiologue","audioprothésiste"
    ],
    "Neurochirurgie": [
        "neurochirurgie"
    ],
    "Dermatologie": [
        "dermatologue","dermatologie", "dermatologue","dermatologue pédiatrique","dermatologie et venereologie",
        "dermatologie pédiatrique"
    ],
    "Nutrition":["diététique","nutritionniste","nutrition"],
    "Endocrinologie": [
        "diabétologue","diététicien","endocrinologie", "endocrinologue", "endocrinologie et métabolismes",
        "diabétologie nutritionnelle"
    ],
    "Gastro-entérologie": [
        "gastro-entérologue","gastro-entérologue","gastroentérologie", "gastrologie", "gastrologue",
        "hépatogastroentérologie","du pancréas et des voies biliaires", "hepatogastroenterologue","Hépato-Gastro-Entérologie"
    ],
    "Gériatrie": [
        "gérontologue - gériatre","gériatrie", "gerontologie", "la gériatrie","gérontologie"
    ],
    "Gynécologie": [
        "gynécologie", "gynécologue", "gynécologie obstétrique", "gynecologie-obstetrique","gynecologue","perineologue"
    ],
    "Hématologie": [
        "hématologie","hématologue"
    ],
    "Hépatologie":[
        "hépatologie","hépatologue"  # si présent explicitement
    ],
    "Infectiologie": [
        "infectiologie","maladies infectieuses" # à adapter selon tes données
    ],
    "Médecine aiguë": [
        "médecine d'urgence","médecin urgentiste",
    ],
    "Médecine du travail": [
        "médecine du travail","médecin du travail"
    ],
    "Biologie médicale":["laboratoire d'analyses de biologie médicale","biologiste medicale","biologiste (laboratoire d'analyse)","biologie médicale"],
    "Médecine légale":[
        "médecin légiste",
    ],
    "Médecine générale": [
        "médecin généraliste","médecine générale", "omnipraticien", "médecine de famille","généraliste","médecin de famille"
    ],
    "Médecine interne": [
        "médecine interne","médecin interniste","médecin du sommeil","angiologie"
    ],
    "Médecine nucléaire": [
        "médecine nucléaire","médecin nucléaire"
    ],
    "Médecine palliative": [
        "médecine palliative","algologue - traitement de la douleur"
    ],
    "Médecine physique": [
        "ergothérapie","orthophoniste","posturologue","podologue","chiropracteur","Kinesitherapeute","médecine physique", 
        "médecine physique et de réadaptation", "mpr","kinésithérapie","orthophonie","Podologie","kinésithérapeute",
        "kinésithérapeute du sport","médecin physique et de réadaptation","ergothérapeute","médecin physique réadaptateur"
        
    ],
    "Médecine préventive": [
        "médecin du sport","ostéopathe","homéopathe","médecine préventive", "santé publique", "médecine du sport", "médecine aéronautique et spatiale","Naturopathie",
        "Allergologie","Naturopathie","Médecine Chinoise(Mtc)","ostéopathie","homéopathie","acupuncture","allergologue","allergologue-pédiatrique",
        "tabacologue","naturopathe","sophrologue"
        
    ],
    "Néonatologie": [
        "néonatologie"
    ],
    "Néphrologie": [
        "néphrologie","centre de dialyse","néphrologue","néphrologue pédiatrique","neurophysiologiste"
    ],
    "Neurologie": [
        "neurologie","neurologue"
    ],
    "Odontologie": [
        "endodontiste","occlusodontiste","orthodontiste","parodontologiste","pédodontiste",
        "dentiste", "chirurgie dentaire", "chirurgie dentiste", "dentaire","implantologiste",
        "orthodontie", "parodontologie", "pédodontie"
    ],
    "Oncologie": [
        "oncologie", "cancérologie","oncologue - cancérologue","oncologue médical","cancérologue","oncologue"
    ],
    "Obstétrique": [
        "obstétrique", "sage femme","sage-femme"
    ],
    "Ophtalmologie": [
        "ophtalmologie", "orthoptie","ophtalmologue","ophtalmologue pédiatrique","orthoptiste","néonatologiste"
    ],
    "Orthopédie": [
        "traumato-orthopédiste pédiatrique","traumatologue-orthopédiste","Orthopédie traumatolo","orthopédie", "traumatologie", "traumato - orthopedie",
        "chirurgie orthopédiste", "chirurgie traumatologie-orthopédie", "chirurgie orthopédique","Perineologue","Orthopédiste Traumatologue"
    ],
    "Oto-rhino-laryngologie": [
        "orl","oto-rhino-laryngologie","oto-rhino-laryngologie(orl)","oto-rhino-laryngologie et chirurgie cervico-faciale"
    ],
    "Pédiatrie ": [
        "pédiatrie","pédiatre"
    ],
    "Anatomopathologie": [
    "anatomopathologiste", "anatomopathologie", "médecin anatomopathologiste"
    ],
    "Aneumologie ": [
        "pneumologie","pneumologue"
    ],
    "Psychiatrie ": [
        "psychiatrie","psychanalyste","coach développement personnel","hypnothérapeute","pédopsychiatre","psychiatre","psychologue","psychologue","psychomotricien","psychothérapeute","sexologue","sophrologue"
        "psychiatrie","sexologie","neuropsychiatrie", "psychologie", "psychomotricité", "psychothérapie", "pedopsychiatrie","Addictologie","psycologue","addictologue"
    ],
    "Radiologie ": [
        "radiologie","radiologue"
    ],
    "Radiothérapie ": [
        "radiothérapeute","radiothérapie", "oncologie radiothérapie", "oncologie option radiothérapie"
    ],
    "Rhumatologie": [
        "rhumatologie","rhumatologue"
    ],
    "Urologie": [
        "urologie", "chirurgie urologique","urologue","urologue pédiatrique"
    ]}
def reclasser_specialite(specialite):
    specialite = str(specialite).strip().lower().replace("nan","")
    for categorie, mots_cles in classification.items():
        for mot in mots_cles:
            if mot.lower() in specialite:
                return categorie
    return specialite if specialite else ""

df1["Spécialité"]=df1["Spécialité"].apply(reclasser_specialite)
df1["Spécialité"]=df1["Spécialité"].astype(str).str.title()
#-------------Nettoyer Ville--------------------------------
df1["Ville"]=df1["Ville"].astype(str).str.title()
df1["Ville"]=(df1["Ville"]
              .str.replace("Temara","Témara")
              .str.replace("Khemisset","Khémisset")
              .str.replace("Benguérir","Ben Guerir")
              .str.replace("Beni Mellal","Béni Mellal")
              .str.replace("Skhirat","Skhirate")
              .str.replace("Skhiratee","Skhirate")
              .str.replace("Fkih Ben Saleh","Fkih Ben Salah")
              .str.replace("Mohammadia","Mohammédia")
              .str.replace("Mohammedia","Mohammédia")
              .str.replace("Mohammadia","Mohammédia"))
#------------Nettoyer Telephone-------------------------------
def Nettoyage_phone(tel):
    if pd.isna(tel):return""
    tel=str(tel).strip().replace("-","").replace(" ","")
    if tel.startswith("0") and len(tel)>1:
        tel="+212"+tel[1:]
    elif tel.startswith("6") or tel.startswith("5")or tel.startswith("7"):
        tel="+212"+tel
    if tel in ["","+212","nan","None","+212nan"]:
        return ""
    return tel
df1["Téléphone"]=df1["Téléphone"].apply(Nettoyage_phone)

#--------------Nettoyage de la date Extraction---------------------
df1["Date Extraction"]=pd.to_datetime("2025-07-28 19:15:22", format="%Y-%m-%d %H:%M:%S")
#--------------Nettoyage des Langues Parlées--------------------------------
df1["Langues Parlées"]=(df1["Langues Parlées"]
                        .astype(str).str.replace("عربي","Arabe")
                        .replace("Nan","")
                        .replace("nan","")
                        .replace("NAN","")
                        .replace("none","")
                         .replace("[","")
                         .replace("]",""))
#-------------------------------------------------------------------------------------------------------------
ListString=["ID Medecin","Civilité","Nom","Prénom"]

for col in ListString:
    df1[col]=(df1[col].
              astype(str)
              .str.replace("-"," ")
              .replace("nan","")
              .replace("Nan","")
              .replace("NAN","")
              .replace("none","")
              .str.replace("عربي", "Arabe")
              .str.replace(r'[\u0600-\u06FF\u0750-\u077F\u08A0-\u08FF]+','', regex=True))
    
Nouvel_Order=["ID Medecin","Statut","Prénom","Nom","Spécialité","Ville","Téléphone","Langues Parlées","Date Extraction"]
df1=df1[Nouvel_Order]
df1.to_excel(r"C:\Users\HP\Desktop\Data_Folder\Fichier_Dabadoc_ma_Apres_Nettoyage.xlsx",index=False)
df1

NameError: name 'df' is not defined

In [21]:
#Insérer les elements pour la collection dabadoc
from ConnexionMongo_DB import get_db
from insertion import inserer_dataframe
db=get_db()
inserer_dataframe(df1,db,"dabadoc")

succes
19275 documents insérés dans 'dabadoc'


In [8]:
#Identifier les elements récemment insérés
date_courante="2025-07-23 13-49-51"
df_nouveau=df1[df1["Date Extraction"]==date_courante]
#Identifier les Ids
ids_ancienA=set(df1[df1["Date Extraction"]<date_courante]["ID Medecin"])
ids_nouveauA=set(df_nouveau["ID Medecin"])
id_ajoutes=ids_nouveauA-ids_ancienA
df_nouveau=df_nouveau[df_nouveau["ID Medecin"].isin(id_ajoutes)]
print(f"le nombre des elements ajoutes{len(df_nouveau)}")

le nombre des elements ajoutes1100


In [ ]:
#Identifier les elements supprimés
date_courante="2025-07-23 13-49-51"
df_ancien=df1[df1["Date Extraction"]<date_courante]
#identifier les ids
ids_ancienS=set(df_ancien["ID Medecin"])
ids_nouveauA=set(df1[df1["Date Extraction"]==date_courante]["ID Medecin"])
id_ajoutes=ids_ancienA-ids_nouveauA

In [74]:
df1["Langues Parlées"].apply(type).unique()

array([<class 'dict'>], dtype=object)